# Merged Flame Graph

How to generate a Flame Graph out of a Chrome profile. This Flame Graph includes both Trace Event traces and Inspector 1.2 profiles.. Example file was recorded in Chrome.

In [25]:
f = open('examples/Profile-20180716T115056', 'r')

In [26]:
import json

profile = json.load(f)

In [27]:
start = None
end = None

profile_events = []
trace_events = []

for row in profile:
    if row['ph'] != 'M':
        if start is None or int(row['ts']) < start:
            start = int(row['ts'])
        if end is None or int(row['ts']) > end:
            end = int(row['ts'])
    if row['ph'] in ['B', 'E', 'X']:
        trace_events.append(row)
    elif row['ph'] == 'I' and row['name'] == 'CpuProfile':
        profile_events.append(row)

In [28]:
def parse_nodes(data):
    nodes = {}
    for node in data['cpuProfile']['nodes']:
        node_id = node['id']
        function_name = node['callFrame']['functionName']
        url = node['callFrame']['url']
        line_number = node['callFrame']['lineNumber']
        children = node.get('children')
        hit_count = node.get('hitCount')
        nodes[node_id] = {'function_name': function_name, 'url': url, 'line_number': line_number, 'hit_count': hit_count, 'children': children}
    return nodes

In [29]:
import copy

def generate_stacks(node_id, nodes, stacks, current_stack):
    node = nodes[node_id] # break in case id doesn't exist
    if node['function_name'] == '':
        node['function_name'] = '(anonymous)'
    if node['function_name'] != '(root)':
        current_stack.append(node['function_name'])
        stacks[node_id] = current_stack
    if node['children']:
        for child in node['children']:
            generate_stacks(child, nodes, stacks, copy.copy(current_stack))
    del nodes[node_id]

In [30]:
def create_begin_events(pid, tid, stack, ts):
    events = []
    # walk forward on the stack
    for frame in stack:
        events.append({
            'pid': pid,
            'tid': tid,
            'name': frame,
            'cat': 'jssample',
            'ph': 'B',
            'ts': ts
        })
    return events
    
def create_end_events(pid, tid, stack, ts):
    events = []
    # walk backwards on the stack
    for frame in reversed(stack):
        events.append({
            'pid': pid,
            'tid': tid,
            'name': frame,
            'cat': 'jssample',
            'ph': 'E',
            'ts': ts
        })
    return events

# TODO: several slices start and end at the same timestamp

def change_stack(pid, tid, previous_stack, current_stack, ts, is_last_sample):
    if is_last_sample:
        # just close everything
        return create_end_events(pid, tid, previous_stack, ts)
    stack_index = 0
    for previous_frame in previous_stack:
        current_frame = None
        if stack_index < len(current_stack):
            current_frame = current_stack[stack_index]
        if current_frame is None:
            # the previous stack is equal to the current
            # but current stack is shorter than previous
            # have to end the remaining of the previous stack
            return create_end_events(pid, tid, previous_stack[stack_index:], ts)
        if current_frame != previous_frame:
            # at this frame, the stacks differ
            # have to end the previous stack from here and begin a new stack
            events = []
            events += create_end_events(pid, tid, previous_stack[stack_index:], ts)
            events += create_begin_events(pid, tid, current_stack[stack_index:], ts)
            return events
        stack_index += 1
    if len(current_stack) > stack_index:
        # the previous stack is equal to the current
        # but the current stack is longer
        return create_begin_events(pid, tid, current_stack[stack_index:], ts) 
    # stack is the same
    return []

In [31]:
def get_events(pid, tid, samples, time_deltas, start_time, stacks, ignore_ids):
    events = []
    current_time = start_time
    previous_sample = None
    previous_stack = []
    for index, sample in enumerate(samples):
        delta = time_deltas[index]
        if delta < 0:
            delta = 0
        current_time += delta
        if sample != previous_sample:
            current_stack = stacks[sample]
            is_last_sample = index == (len(samples) - 1)
            if sample in ignore_ids:
                current_stack = []
            events += change_stack(pid, tid, previous_stack, current_stack, current_time, is_last_sample)
            previous_sample = sample
            previous_stack = current_stack
    return events

In [32]:
def get_meta_ids(nodes):
    program_node_id = None
    idle_node_id = None
    gc_node_id = None
    for key, node in nodes.items():
        if node['function_name'] == '(program)':
            program_node_id = key
        elif node['function_name'] == '(idle)':
            idle_node_id = key
        elif node['function_name'] == '(garbage collector)':
            gc_node_id = key
    return program_node_id, idle_node_id, gc_node_id

In [33]:
v8_events = []

for profile in profile_events:
    pid = profile['pid']
    tid = profile['tid']
    data = profile['args']['data']
    root_id = data['cpuProfile']['nodes'][0]['id']
    nodes = parse_nodes(data)
    ignore_ids = get_meta_ids(nodes)
    stacks = {}
    generate_stacks(root_id, nodes, stacks, [])
    v8_events += get_events(pid, tid, data['cpuProfile']['samples'], data['cpuProfile']['timeDeltas'], data['cpuProfile']['startTime'], stacks, ignore_ids)


In [35]:
# from sortedcollection import SortedCollection
# from operator import itemgetter

# sorted_profile = SortedCollection(key=itemgetter('ts'))

# for event in trace_events: 
#     sorted_profile.insert_right(event)

# for event in js_events:
#     sorted_profile.insert(event)

In [36]:
# TODO: better merging mechanism

trace_events_index = 0

for v8_event in v8_events:
    for index, trace_event in enumerate(trace_events[trace_events_index:]):
        

In [34]:
root = {'name': 'root', 'value': 0, 'children': []}
open_partial_slices = {}

# TODO: handle CPU time differences, where "E" comes before "B"

def get_child_slice(parent_slice, name):
    for index, child in enumerate(parent_slice['children']):
        if child['name'] == name:
            return parent_slice['children'].pop(index)
    return None

def insert_slice(parent_slice, new_slice):
    child_slice = get_child_slice(parent_slice, new_slice['name'])
    if child_slice is None:
        child_slice = {'name': new_slice['name'], 'value': 0, 'children': []}
    for child in new_slice['children']:
        insert_slice(child_slice, child)
    child_slice['value'] += new_slice['value']
    parent_slice['children'].append(child_slice)

def check_thread(pid, tid):
    if pid not in open_partial_slices:
        open_partial_slices[pid] = {}
    if tid not in open_partial_slices[pid]:
        open_partial_slices[pid][tid] = []

def begin_slice(pid, tid, cat, name, ts, tts):
    check_thread(pid, tid)
    open_partial_slices[pid][tid].append({'pid': pid, 'tid': tid, 'cat': cat, 'name': name, 'ts': ts, 'tts': tts, 'children': []})

def end_slice(pid, tid, cat, name, ts, tts):
    partial_slice_count = len(open_partial_slices[pid][tid])
    if partial_slice_count > 0:
        current_slice = open_partial_slices[pid][tid].pop()
        if current_slice['cat'] != cat or current_slice['name'] != name:
            raise Exception("ending slice with different cat or name", pid, tid, cat, name, ts)
        current_slice['dur'] = ts - current_slice['ts']
        if tts is not None and current_slice['tts'] is not None:
            current_slice['tdur'] = tts - current_slice['tts']
        if 'tdur' in current_slice:
            current_slice['value'] = current_slice['tdur']
        else:
            current_slice['value'] = current_slice['dur']
        partial_slice_count = len(open_partial_slices[pid][tid])
        if partial_slice_count > 0:
            open_partial_slices[pid][tid][partial_slice_count - 1]['children'].append(current_slice)
        else:
            insert_slice(root, current_slice)
    else:
        raise Exception("end_slice called without an open slice", pid, tid, cat, name, ts)

In [37]:
# TODO: handle "sf" and "stack" properties on Duration Events

for row in iter(trace_events):
    if row['ph'] == 'B' or row['ph'] == 'E':
        if row['ph'] == 'B':
            begin_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'], row.get('tts'))
        elif row['ph'] == 'E':
            end_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'], row.get('tts'))
    elif row['ph'] == 'X':
        if 'tts' in row and 'tdur' in row:
            end_tts = row['tts'] + row['tdur']
        if 'dur' in row and row['dur'] > 0:
            begin_slice(row['pid'], row['tid'], row['cat'], row['name'], row['ts'], row.get('tts'))
            end_slice(row['pid'], row['tid'], row['ts'] + row['dur'], end_tts)

In [38]:
import json

with open('merged.json', 'w') as file:
     file.write(json.dumps(root))